# MOSTLY AI vs. SDV Comparison - Referential Integrity Scenario  <a href="https://colab.research.google.com/github/mostly-ai/mostlyai/blob/main/docs/tutorials/sdv-comparison/referential-integrity-scenario/referential-integriry-scenario.ipynb" target="_blank"><img src="https://img.shields.io/badge/Open%20in-Colab-blue?logo=google-colab" alt="Run on Colab"></a>

This notebook provides a comparison of the ability to preserve referential integrity between two leading synthetic data generation platforms:
- **SDV (Synthetic Data Vault)** - Business Source License
- **MOSTLY AI Synthetic Data SDK** - Apache 2.0 License - Open Source

In this comparison, we are going to walk through the synthesis of a relational two-table structure using the [Berka dataset](https://github.com/mostly-ai/public-demo-data/tree/dev/berka/data).

## Comparison Methodology

1. **Data Preparation**: Load, inspect, and preprocess the multi-table dataset
2. **Data Splitting**: Create train/test splits that maintain referential integrity
3. **Model Training**: Train both SDV and MOSTLY AI generators on the training data
4. **Synthetic Data Generation**: Generate synthetic datasets using both platforms
5. **Performance Analysis**: Compare training time, generation speed, and data quality

## Key Challenges in Multi-table Synthesis

- **Referential Integrity**: Preserving temporal patterns in transaction data
- **Data Quality**: Ensuring synthetic data maintains statistical properties and business logic

In [ ]:
# Install SDK in CLIENT mode
!uv pip install -U mostlyai
# Or install in LOCAL mode
!uv pip install -U 'mostlyai[local]'  
# Note: Restart kernel session after installation!

!uv pip install -q scikit-learn seaborn lightgbm sdv

## 1. Data Loading and Initial Exploration

First, let's load our dataset and examine its structure to understand:
- Table schemas and data types
- Relationships between tables
- Business logic and constraints


In [ ]:
import pandas as pd


# GLIEF dataset hosted by MOSTLY AI
base_url = "https://raw.githubusercontent.com/mostly-ai/public-demo-data/dev/gleif/"

originals = {
    "organizations": pd.read_csv(base_url + "organizations.csv.gz", compression="gzip", low_memory=False),
    "relations": pd.read_csv(base_url + "relations.csv.gz", compression="gzip", low_memory=False),
}

# Print samples
for k, df in originals.items():
    print("===", k, "=== shape:", df.shape)
    display(df.sample(n=3))


In [ ]:
# Build dataframes
df_organizations = originals["organizations"].copy()
df_relations     = originals["relations"].copy()

# Print shapes
print("\n📋 Final dataframe shapes:")
print(f"   - Organizations: {df_organizations.shape[0]:,} × {df_organizations.shape[1]}")
print(f"   - Relations:     {df_relations.shape[0]:,} × {df_relations.shape[1]}")


## 2. Strategic Data Splitting for Multi-Table Scenarios

When dealing with multiple related tables, splitting data becomes more complex than simple random sampling. 

We need to consider:

- **Referential Integrity**: Ensure foreign key relationships remain valid in both splits
- **Business Logic**: Relations can only exist between organizations in the same split
- **Data Leakage Prevention**: Avoid information bleeding between train/test sets

**Our Approach:**
1. Split organizations first (80/20 train/test)
2. Assign relations based on participant customers
3. Relations go to training set only if both `START_ID` and `END_ID` are in training set
4. All other relations go to test set


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

ORG_KEY = "ID"
REL_START = "START_ID"
REL_END = "END_ID"

print("Performing multi-table split (organizations / relations)...")

# Dtype alignment to avoid false negatives
if df_relations[REL_START].dtype != df_organizations[ORG_KEY].dtype:
    try:
        df_relations[REL_START] = df_relations[REL_START].astype(df_organizations[ORG_KEY].dtype)
        df_relations[REL_END]   = df_relations[REL_END].astype(df_organizations[ORG_KEY].dtype)
    except Exception:
        df_organizations[ORG_KEY] = df_organizations[ORG_KEY].astype(df_relations[REL_START].dtype)

# Split organizations 80/20
org_train, org_test = train_test_split(
    df_organizations, test_size=0.2, random_state=42, stratify=None
)

print("Organizations split:")
print(f"  - Training set: {len(org_train):,} orgs ({len(org_train)/len(df_organizations)*100:.1f}%)")
print(f"  - Test set:     {len(org_test):,} orgs ({len(org_test)/len(df_organizations)*100:.1f}%)")

train_ids = set(org_train[ORG_KEY])
test_ids  = set(org_test[ORG_KEY])

# Sense checks
overlap_ids = train_ids & test_ids
if overlap_ids:
    raise ValueError(f"Split error: {len(overlap_ids)} overlapping {ORG_KEY} values between train/test.")

all_ids = train_ids | test_ids
missing_from_split = set(df_organizations[ORG_KEY]) - all_ids
if missing_from_split:
    raise ValueError(f"{len(missing_from_split)} {ORG_KEY} values not included in either split.")

rel = df_relations.copy()

in_train = rel[REL_START].isin(train_ids) & rel[REL_END].isin(train_ids)
in_test  = rel[REL_START].isin(test_ids)  & rel[REL_END].isin(test_ids)

relations_train = rel[in_train].copy()
relations_test  = rel[in_test].copy()

print("Relations split (edges with both endpoints in the same partition):")
print(f"  - Training relations: {len(relations_train):,} "
      f"({len(relations_train)/len(df_relations)*100:.1f}%)")
print(f"  - Test relations:     {len(relations_test):,} "
      f"({len(relations_test)/len(df_relations)*100:.1f}%)")

unknown_nodes = set(pd.concat([rel[REL_START], rel[REL_END]]).unique()) - set(df_organizations[ORG_KEY])
if unknown_nodes:
    raise ValueError(f"Found {len(unknown_nodes)} relation node IDs not present in organizations.")

org_train_out  = "./data/organizations_train.parquet"
org_test_out   = "./data/organizations_test.parquet"
rel_train_out  = "./data/relations_train.parquet"
rel_test_out   = "./data/relations_test.parquet"

org_train.to_parquet(org_train_out, index=False)
org_test.to_parquet(org_test_out, index=False)
relations_train.to_parquet(rel_train_out, index=False)
relations_test.to_parquet(rel_test_out, index=False)


## 3. SDV (Synthetic Data Vault) Implementation

**About SDV:**
- Business Source License Python library for synthetic data generation
- Supports single-table and multi-table scenarios
- Uses statistical modeling and machine learning approaches
- Provides HMASynthesizer for hierarchical multi-table synthesis

**Key Features:**
- **Metadata Detection**: Automatically infers data types and relationships
- **Relationship Modeling**: Handles parent-child table relationships
- **Privacy Protection**: Generates synthetic data that preserves statistical properties while protecting individual privacy
- **Extensible**: Multiple synthesizer options (GaussianCopula, CTGAN, CopulaGAN, etc.)

**Limitations:**
- Current version only supports one parent per child table
- Complex multi-parent relationships require modeling simplification
- Performance scales with data complexity


### 3.1 SDV Metadata Configuration

The relations table has TWO foreign keys (`START_ID` and `END_ID`) both referencing the organizations table. 

SDV v1.x only supports one parent per child, so we'll model one relationship explicitly.


In [ ]:
from sdv.multi_table import HMASynthesizer
from sdv.metadata import Metadata

metadata = Metadata.detect_from_dataframes(
    data={
        'organizations': org_train,
        'relations': relations_train
    },
    infer_keys='primary_and_foreign'
)

metadata.add_relationship(
    parent_table_name='organizations',
    child_table_name='relations',
    parent_primary_key='ID',
    child_foreign_key='START_ID'
)

print("\n📋 SDV Metadata Configuration:")
metadata


### 3.2 SDV Model Training

**HMASynthesizer Overview:**
- **Hierarchical Modeling**: Learns parent-child relationships
- **Statistical Approach**: Uses copulas and Gaussian distributions
- **Multi-step Process**: 
  1. Preprocesses tables and infers constraints
  2. Learns relationships between parent and child tables
  3. Models individual table distributions
  
**Training Phases:**
- **Preprocess Tables**: Data cleaning and type inference
- **Learning Relationships**: Analyzing foreign key dependencies  
- **Modeling Tables**: Learning statistical distributions for synthesis


In [ ]:
import time

start_time = time.time()

# Initialize HMASynthesizer
print("🔧 Initializing HMASynthesizer...")
synthesizer = HMASynthesizer(metadata)

# Training configuration
synthesizer.fit({
    'organizations': org_train,
    'relations': relations_train
})

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

print(f"⏱️ Total training time: {elapsed_minutes:.2f} minutes")
print(f"📈 Training data: {len(org_train):,} organizations, {len(relations_train):,} relations")


### 3.3 SDV Synthetic Data Generation

**Generation Process:**
- **Scale Parameter**: Controls the number of synthetic records (1.0 = same size as training data)
- **Hierarchical Generation**: First generates parent records (customers), then child records (transfers)
- **Relationship Preservation**: Ensures all synthetic transfers reference valid synthetic customers
- **Statistical Sampling**: Uses learned distributions to create realistic synthetic data

We'll create just 10% of the total number of records to reduce generation time.


In [ ]:
import time

start_time = time.time()

sdv_synthetic_data = synthesizer.sample(scale=0.25)

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

# Performance metrics
synthetic_orgs_count = len(sdv_synthetic_data['organizations'])
synthetic_rels_count = len(sdv_synthetic_data['relations'])
total_synthetic_records = synthetic_orgs_count + synthetic_rels_count
generation_rate = total_synthetic_records / (end_time - start_time)

print(f"⏱️  Generation time: {elapsed_minutes:.2f} minutes")
print(f"🚀 Generation rate: {generation_rate:,.0f} records/second")
print(f"📊 Generated {synthetic_orgs_count:,} synthetic organizations")
print(f"📊 Generated {synthetic_rels_count:,} synthetic relations")

# Data quality verification
relations_per_org = (
    synthetic_rels_count / synthetic_orgs_count if synthetic_orgs_count > 0 else 0
)

print(f"\n Generation Quality Metrics:")
print(f"   - Relations per organization: {relations_per_org:.1f}")
print(f"   - Scale factor achieved: {synthetic_orgs_count / len(org_train):.2f}x")


In [ ]:
# Save SDV synthetic data for comparison
output_folder = './data/referential-integrity/'

synthetic_files = {
    'organizations': f'{output_folder}sdv_organizations.parquet',
    'relations': f'{output_folder}sdv_relations.parquet',
}

for table_name, file_path in synthetic_files.items():
    sdv_synthetic_data[table_name].to_parquet(file_path, index=False)
    print(f"💾 Saved {table_name} synthetic data to: {file_path}")


## 4. MOSTLY AI Implementation

**About MOSTLY AI Synthetic Data SDK:**
- Open-source (Apache 2 license) synthetic data SDK with advanced AI capabilities
- Also cloud-based service with enterprise-grade security and compliance
- Supports complex multi-table scenarios with multiple foreign keys

**Key Advantages:**
- **Advanced AI Models**: Utilizes state-of-the-art generative AI including language models
- **Multi-Parent Support**: Can handle complex relationships (multiple foreign keys per table)
- **Mixed Data Types**: Excels at both tabular and text data synthesis


In [ ]:
from mostlyai.sdk import MostlyAI

# Initialize MOSTLY AI Synthetic Data SDK
mostly = MostlyAI(local=True)

### 4.1 MOSTLY AI Configuration

As before, the relations table has TWO foreign keys (`START_ID` and `END_ID`) both referencing the organizations table. 

MOSTLY AI supports multiple foreign keys referencing our subject table (organizations).

In [ ]:
config = {
    'name': 'GLEIF Organizations & Relations Generator',
    'tables': [
        {
            'name': 'organizations',
            'data': org_train,
            'primary_key': 'ID',  # first column in organizations
            'tabular_model_configuration': {
                'enable_model_report': False
            }
        },
        {
            'name': 'relations',
            'data': relations_train,
            'foreign_keys': [
                {
                    'column': 'START_ID',
                    'referenced_table': 'organizations',
                    'referenced_column': 'ID',
                    'is_context': True
                },
                {
                    'column': 'END_ID',
                    'referenced_table': 'organizations',
                    'referenced_column': 'ID',
                    'is_context': False
                }
            ],
            'tabular_model_configuration': {
                'enable_model_report': False
            }
        }
    ]
}



### 4.2 MOSTLY AI Training Process

**Training Process:**
1. **Upload Data**: Send training data securely to MOSTLY AI cloud
2. **Model Configuration**: Apply the complex multi-table configuration
3. **AI Training**: Use advanced generative models including LLMs
4. **Quality Validation**: Automatic quality checks during training

**TabularARGN**
- Flexible and efficient auto-regressive framework for generating high-fidelity synthetic data
- Arxiv paper: https://arxiv.org/abs/2501.12012



In [ ]:
start_time = time.time()

# Train MOSTLY AI generator with our configuration
g = mostly.train(config=config, start=True, wait=True)

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

print(f"⏱️ Total training time: {elapsed_minutes:.2f} minutes")
    

In [ ]:
import time

TARGET_FRACTION = 0.10
target_size = max(1, int(len(org_train) * TARGET_FRACTION))

start_time = time.time()

print(f"📊 Generating {target_size:,} synthetic organization records "
      f"(~{TARGET_FRACTION:.0%} of {len(org_train):,})...")

# Generate synthetic data
sd = mostly.generate(g, size=target_size)
mostlyai_synthetic_data = sd.data()

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

# Calculate generation statistics
total_records = sum(len(mostlyai_synthetic_data[table]) for table in mostlyai_synthetic_data.keys())
generation_rate = total_records / (end_time - start_time)

# Performance metrics
print(f"⏱️ Generation time: {elapsed_minutes:.2f} minutes")
print(f"🚀 Generation rate: {generation_rate:,.0f} records/second")

print("📊 Synthetic data breakdown:")
for table_name in mostlyai_synthetic_data:
    print(f"   - {table_name.capitalize()}: {len(mostlyai_synthetic_data[table_name]):,} rows")
    


In [ ]:
# Save MOSTLY AI synthetic data for comparison
output_folder = './data/referential-integrity/'

mostlyai_files = {
    'organizations': f'{output_folder}mostlyai_organizations.parquet',
    'relations': f'{output_folder}mostlyai_relations.parquet',
}

for table_name, file_path in mostlyai_files.items():
    mostlyai_synthetic_data[table_name].to_parquet(file_path, index=False)
    print(f"💾 Saved {table_name} synthetic data to: {file_path}")


## 5. Synthetic Data Quality Assessment

After generating synthetic data using both SDV and MOSTLY AI, it's crucial to comprehensively evaluate the quality, privacy, and integrity of the generated datasets. This section provides a multi-faceted quality assessment framework that ensures our synthetic data meets production standards.

### **Quality Assessment Framework:**

Our evaluation methodology consists of two complementary approaches:

1. **Statistical Quality Assessment (Section 5.1)**: Using the MOSTLY AI QA library to evaluate statistical fidelity, privacy metrics, and overall data quality for the relations table (the subject of this experiment)
2. **Foreign Key Integrity Verification (Section 5.2)**: Custom verification to ensure referential integrity and relationship preservation in multi-table synthetic data

### **Key Quality Dimensions Evaluated:**

- **Statistical Accuracy**: How well synthetic data preserves statistical properties of the original data
- **Privacy Protection**: Measurement of privacy risks and distance to closest record (DCR)
- **Referential Integrity**: Verification that foreign key relationships are maintained correctly
- **Utility vs Privacy Balance**: Evaluation of the trade-off between data utility and privacy protection

## 5.1 Statistical Quality Assessment with MOSTLY AI QA Library

The MOSTLY AI QA library provides enterprise-grade quality assessment capabilities that evaluate synthetic data across multiple dimensions. This assessment generates comprehensive HTML reports and quantitative metrics that help understand:

- **Accuracy Scores**: Overall statistical fidelity of synthetic data
- **Distance to Closest Record (DCR)**: Privacy risk measurement 
- **Univariate & Bivariate Distributions**: Preservation of individual column and column-pair statistics
- **Correlation Analysis**: Maintenance of relationships between variables
- **Similarity Metrics**: Overall resemblance to training data while avoiding overfitting

These assessments are performed for the target dataset - relations, comparing synthetic data against both training and holdout datasets to ensure robust evaluation.

In [ ]:
# Import and initialize the quality assessment framework
from mostlyai import qa

# Optionally run logging for a more detailed output
# qa.init_logging()

In [ ]:
import pandas as pd

# Load the split files from disk
relations_train = pd.read_parquet('./data/relations_train.parquet')
relations_test  = pd.read_parquet('./data/relations_test.parquet')

print("✅ Loaded splits:")
print(f"  - relations_train: {relations_train.shape}")
print(f"  - relations_test:  {relations_test.shape}")


## Evaluate Relations

The relations table contains the `START_ID` and `END_ID` fields, which reference the organizations table. Our ultimate target output will show not only a statistically accurate organizations table, but one with entirely valid foreign key relations.

In [ ]:
import pandas as pd

# Load the split files from disk
relations_train = pd.read_parquet('./data/relations_train.parquet')
relations_test  = pd.read_parquet('./data/relations_test.parquet')

print("✅ Loaded splits:")
print(f"  - org_train:       {org_train.shape}")
print(f"  - org_test:        {org_test.shape}")
print(f"  - relations_train: {relations_train.shape}")
print(f"  - relations_test:  {relations_test.shape}")

# Load the SDV synthetic dataset
sdv_relations = pd.read_parquet('./data/referential-integrity/sdv_relations.parquet')

# Run QA with organizations as context table
report_path, metrics = qa.report(
    syn_tgt_data = sdv_relations,
    trn_tgt_data = relations_train,
    hol_tgt_data = relations_test,
    syn_ctx_data = pd.read_parquet('./data/referential-integrity/sdv_organizations.parquet'),
    trn_ctx_data = org_train,
    hol_ctx_data = org_test,
    ctx_primary_key = "ID",
    tgt_context_key = "START_ID",   # modelled FK in SDV
    max_sample_size_embeddings=10_000,
    report_path='sdv_relations_qa_report.html'
)

print(f"📋 SDV Relations Quality Report saved to: {report_path}")
print("\n📈 SDV Relations Quality Metrics:")
print(metrics.model_dump_json(indent=4))

# Print summary scores
sdv_relations_accuracy = metrics.accuracy.overall
sdv_relations_dcr_share = metrics.distances.dcr_share
print(f"\n🎯 SDV Relations Summary:")
print(f"   Overall Accuracy: {sdv_relations_accuracy:.3f}")
print(f"   DCR Share: {sdv_relations_dcr_share:.3f}")


In [ ]:
syn_rel_path = './data/referential-integrity/mostlyai_relations.parquet'
syn_org_path = './data/referential-integrity/mostlyai_organizations.parquet'
syn_rel = pd.read_parquet(syn_rel_path).copy()
syn_org = pd.read_parquet(syn_org_path).copy()

# Cast dtypes to strings
for df, kind in [(syn_rel, "syn_rel"), (relations_train, "trn_rel"), (relations_test, "hol_rel")]:
    for col in ("START_ID", "END_ID", "RELATION_ID"):
        if col in df.columns:
            df[col] = df[col].astype("string")
    for col in ("TYPE", "STATUS"):
        if col in df.columns:
            df[col] = df[col].astype("string")

for df, kind in [(syn_org, "syn_org"), (org_train, "trn_org"), (org_test, "hol_org")]:
    if "ID" in df.columns:
        df["ID"] = df["ID"].astype("string")
    for col in ("STATUS", "LEICATEGORY_NAME", "COUNTRY_NAME"):
        if col in df.columns:
            df[col] = df[col].astype("string")

# Diagnostics
total_syn_rel = len(syn_rel)
non_null_start = syn_rel["START_ID"].notna().sum() if "START_ID" in syn_rel else 0
matched_to_ctx = syn_rel["START_ID"].isin(syn_org["ID"]).sum() if "START_ID" in syn_rel and "ID" in syn_org else 0

print(f"🧮 Synthetic relations: {total_syn_rel:,} rows "
      f"(START_ID non-null: {non_null_start:,}, START_ID matching ctx.ID: {matched_to_ctx:,})")

MIN_ROWS = 100
if matched_to_ctx < MIN_ROWS:
    print("⚠️ Not enough usable synthetic rows for QA "
          f"(need ≥ {MIN_ROWS}, have {matched_to_ctx}).")
    print("👉 Check that you’re loading the correct synthetic files and that keys match the context:")
    print(f"   - syn_rel path: {syn_rel_path}")
    print(f"   - syn_org path: {syn_org_path}")
    print("   - If you recently generated only a tiny sample, regenerate with a larger size for relations/organizations.")
else:
    # Run QA
    report_path, metrics = qa.report(
        syn_tgt_data=syn_rel,
        trn_tgt_data=relations_train,
        hol_tgt_data=relations_test,
        syn_ctx_data=syn_org,
        trn_ctx_data=org_train,
        hol_ctx_data=org_test,
        ctx_primary_key="ID",
        tgt_context_key="START_ID",   # one linkage at a time
        max_sample_size_embeddings=1000,
        report_path='mostlyai_relations_qa_report.html'
    )
    print(f"📋 MOSTLY AI Relations Quality Report saved to: {report_path}")

    if metrics is None:
        print("⚠️ QA returned no metrics (insufficient rows after internal filtering).")
    else:
        print("\n📈 MOSTLY AI Relations Quality Metrics:")
        print(metrics.model_dump_json(indent=4))
        # Summary
        print("\n🎯 MOSTLY AI Relations Summary:")
        print(f"   Overall Accuracy: {metrics.accuracy.overall:.3f}")
        print(f"   DCR Share: {metrics.distances.dcr_share:.3f}")


## 6.2 Foreign Key Integrity Verification

One critical aspect of multi-table synthetic data quality is ensuring that foreign key relationships are maintained properly. This section verifies that:

1. **Referential Integrity**: All foreign keys in synthetic transfers reference valid customer IDs
2. **Coverage**: All synthetic customers are properly referenced in the transfers table
3. **Relationship Patterns**: The distribution of transfers per customer matches expected patterns

This verification is essential for downstream applications that rely on proper table relationships.

In [ ]:
import pandas as pd

def verify_fk_integrity_orgs_relations(
    org_df: pd.DataFrame,
    rel_df: pd.DataFrame,
    provider_name: str,
    pk_col: str = "ID",
    fk_cols: tuple = ("START_ID", "END_ID"),
):
    """
    Comprehensive foreign key integrity verification for GLEIF-style graph (two FKs).

    Args:
        org_df: Synthetic organizations dataframe (parent/nodes)
        rel_df: Synthetic relations dataframe (child/edges)
        provider_name: Name of the synthetic data provider (e.g., 'SDV', 'MOSTLY AI')
        pk_col: Primary key column in org_df (default 'ID')
        fk_cols: Tuple of foreign key columns in rel_df (default ('START_ID','END_ID'))

    Returns:
        Dictionary with integrity and coverage metrics.
    """
    print(f"\n🏷️  {provider_name} Foreign Key Verification (relations → organizations on {fk_cols})")
    print("-" * 60)

    missing_org = [c for c in [pk_col] if c not in org_df.columns]
    missing_rel = [c for c in fk_cols if c not in rel_df.columns]
    if missing_org or missing_rel:
        raise KeyError(
            f"Missing required columns. organizations missing={missing_org}; relations missing={missing_rel}"
        )

    org = org_df.copy()
    rel = rel_df.copy()

    # Align FK dtypes to PK dtype (fallback to string if needed)
    pk_dtype = org[pk_col].dtype
    for fk in fk_cols:
        try:
            rel[fk] = rel[fk].astype(pk_dtype)
        except Exception:
            # fallback: cast all to string
            org[pk_col] = org[pk_col].astype(str)
            rel[fk] = rel[fk].astype(str)
            pk_dtype = org[pk_col].dtype  # update dtype

    # Build sets for checks
    org_ids = set(org[pk_col].dropna().unique())
    fk_sets = {fk: set(rel[fk].dropna().unique()) for fk in fk_cols}

    # Invalid FKs per column
    invalid_per_fk = {fk: fk_sets[fk] - org_ids for fk in fk_cols}
    total_invalid = sum(len(invalid_per_fk[fk]) for fk in fk_cols)

    # Edge-level validity masks
    valid_masks = {fk: rel[fk].isin(org_ids) for fk in fk_cols}
    both_valid_mask = valid_masks[fk_cols[0]] & valid_masks[fk_cols[1]]
    any_invalid_mask = ~(both_valid_mask)

    both_valid_edges = int(both_valid_mask.sum())
    total_edges = int(len(rel))
    any_invalid_edges = int(any_invalid_mask.sum())
    pct_both_valid = (both_valid_edges / total_edges * 100) if total_edges else 0.0
    pct_any_invalid = (any_invalid_edges / total_edges * 100) if total_edges else 0.0

    # Coverage: how many organizations are referenced by at least one endpoint
    referenced_orgs = set(rel.loc[valid_masks[fk_cols[0]], fk_cols[0]].dropna()) | \
                      set(rel.loc[valid_masks[fk_cols[1]], fk_cols[1]].dropna())
    coverage_count = len(referenced_orgs)
    total_orgs = len(org_ids)
    coverage_pct = (coverage_count / total_orgs * 100) if total_orgs else 0.0

    # Degree distribution (count edges touching each org across BOTH endpoints)
    # Build a single Series of all endpoints (valid or not), then count
    endpoints = pd.concat([rel[fk_cols[0]], rel[fk_cols[1]]], ignore_index=True)
    degree_counts = endpoints.value_counts(dropna=True)
    avg_degree = float(degree_counts.mean()) if not degree_counts.empty else 0.0
    median_degree = float(degree_counts.median()) if not degree_counts.empty else 0.0
    max_degree = int(degree_counts.max()) if not degree_counts.empty else 0
    min_degree = int(degree_counts.min()) if not degree_counts.empty else 0

    referential_integrity_all = (total_invalid == 0) and (any_invalid_edges == 0)

    # Results Summary
    metrics = {
        "provider": provider_name,
        "pk_col": pk_col,
        "fk_cols": list(fk_cols),
        "total_organizations": total_orgs,
        "total_relations": total_edges,
        "coverage_referenced_orgs": coverage_count,
        "coverage_percentage": coverage_pct,
        "invalid_fk_values_per_column": {fk: len(invalid_per_fk[fk]) for fk in fk_cols},
        "total_edges_both_valid": both_valid_edges,
        "total_edges_any_invalid": any_invalid_edges,
        "pct_edges_both_valid": pct_both_valid,
        "pct_edges_any_invalid": pct_any_invalid,
        "referential_integrity_all": referential_integrity_all,
        "degree_avg": avg_degree,
        "degree_median": median_degree,
        "degree_max": max_degree,
        "degree_min": min_degree,
    }

    # --- Print report ---
    print("📊 Dataset Summary:")
    print(f"   • Total Organizations: {total_orgs:,}")
    print(f"   • Total Relations:     {total_edges:,}")

    print("\n🔗 Referential Integrity (per FK column):")
    for fk in fk_cols:
        cnt = len(invalid_per_fk[fk])
        status = "✅ PASSED" if cnt == 0 else "❌ FAILED"
        print(f"   • {fk}: {status} — {cnt:,} invalid value(s)")

    print("\n🧮 Edge-Level Integrity:")
    print(f"   • Edges with BOTH endpoints valid: {both_valid_edges:,} ({pct_both_valid:.1f}%)")
    print(f"   • Edges with ANY invalid endpoint: {any_invalid_edges:,} ({pct_any_invalid:.1f}%)")

    print("\n📈 Coverage (nodes referenced by ≥1 edge):")
    print(f"   • Referenced Organizations: {coverage_count:,} ({coverage_pct:.1f}%)")
    print(f"   • Unreferenced Organizations: {max(total_orgs - coverage_count, 0):,}")

    print("\n📊 Node Degree Distribution (edges per organization, counting both ends):")
    print(f"   • Avg: {avg_degree:.2f}")
    print(f"   • Median: {median_degree:.0f}")
    print(f"   • Max: {max_degree:,}")
    print(f"   • Min: {min_degree:,}")

    return metrics

# SDV Synthetic Data
sdv_orgs = pd.read_parquet('./data/referential-integrity/sdv_organizations.parquet')
sdv_rels = pd.read_parquet('./data/referential-integrity/sdv_relations.parquet')

# MOSTLY AI Synthetic Data
mostlyai_orgs = pd.read_parquet('./data/referential-integrity/mostlyai_organizations.parquet')
mostlyai_rels = pd.read_parquet('./data/referential-integrity/mostlyai_relations.parquet')

# Run FK integrity checks
sdv_metrics = verify_fk_integrity_orgs_relations(
    org_df=sdv_orgs,
    rel_df=sdv_rels,
    provider_name="SDV",
    pk_col="ID",
    fk_cols=("START_ID", "END_ID"),
)

mostlyai_metrics = verify_fk_integrity_orgs_relations(
    org_df=mostlyai_orgs,
    rel_df=mostlyai_rels,
    provider_name="MOSTLY AI",
    pk_col="ID",
    fk_cols=("START_ID", "END_ID"),
)
